<a href="https://colab.research.google.com/github/ranvirsahota/AiCore/blob/advanced-python/7-decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decorators


## Lesson Materials
For this lesson, we will use a folder name `utils`. If you are in Colab and currently do not have this folder, run the following code to download the folder with the examples. Remember that you can access `.py` files in Colab and modify them.

In [ ]:
!wget "https://aicore-files.s3.amazonaws.com/Foundations/Python_Programming/advanced_py.zip"
import zipfile
with zipfile.ZipFile("advanced_py.zip", 'r') as zip_ref:
    zip_ref.extractall("utils")

## Introduction
To understand decorators, a basic understanding of inner functions is required.

In [ ]:
def caller(num):

    def first_child():
        print('I am being called')
        return "I am the first child"

    def second_child():
        return "I am the second child"

    if num == 1:
        return first_child
    else:
        return second_child


print(caller(1))

Observe the output of the above code. It returns a function rather than a string or a number. The flow of the code can be explained as follows:

1. The caller function is called with 1 passed as the argument.
2. Thereafter, within the function, two functions are defined: first_child and second_child. However, they are not called; thus, they are not executed.
3. The execution of the caller function proceeds, and the if statement is triggered, which determines if the argument is 1.
4. Finally, only the function is returned, not the result of the function.

> <font size=+1>Decorators extend the functionality of a function.</font>

Consider the code below. First, a function with a nested function is defined. Note that the name of the outer function is decided by the programmer. However, as a standard, the inner function is called `wrapper`.

In this case, a function is defined that accepts another function as an argument. The inner function is defined but not called, following which the inner function is returned.

In [ ]:
def my_decorator(func):
    def wrapper():
        print("I come before the function!")
        func()
        print("I come after the function!")
    return wrapper

def say_truth():
    print(f'Epstein didn\'t kill himself')

Thus, if the outer function is called, the inner function is returned, but not its result.

In [ ]:
my_decorator(say_truth)

The result, `<function __main__.my_decorator.<locals>.wrapper()>`, reveals that the output of the outer function is the inner function.

To improve clarity, let us assign that function to a variable and subsequently call it.

In [ ]:
my_func = my_decorator(say_truth)
my_func()

The output of the inner function is printed above. Basically, `my_func` contains the information about `wrapper` and `say_truth`; therefore, once called, the interaction between `wrapper` and `say_truth` becomes clear.

Consider the below example, where the same decorator is applied with another function.

In [ ]:
def wave():
    print('Hello, world!')

In [ ]:
my_wave_func = my_decorator(wave)
my_wave_func()

Observe that `my_wave_func` contains information about `wrapper` and `wave`, indicating that we can obtain information on the interaction between `wrapper` and `wave`.

## Decorators with Arguments


Thus far, we have discussed decorators that accept no arguments. However, what happens when an argument is passed? Consider the example below, where a function with an argument is passed to the current decorator.

In [ ]:
def say_hello(name):
    print(f'Hello, {name}!')

In [ ]:
my_hello_func = my_decorator(say_hello)

Thus far, there are no errors. In the next step, we call the function.

In [ ]:
my_hello_func()

As shown above, '`I come before the function`' is printed, followed by a type error. Recall that `func()` is the function passed to the decorator (in this case, `say_hello`) which, in turn, expects an argument. In other words, it appears as though the following code is present in `my_decorator`:

``` python
    def wrapper():
        print("I come before the function!")
        say_hello()
        print("I come after the function!")
```

However, `say_hello` requires an argument, hence the error thrown.

Thus, the solution is to format the decorator to expect an argument from the caller, as follows:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("I come before the function!")
        func(name)
        print("I come after the function!")
    return wrapper

my_hello_func = my_decorator(say_hello)

Observe what happens when the code is run:

In [ ]:
my_hello_func()

Python throws a NameError. Notice that the function is in the local scope, where the variable, `name`, has not been defined. Thus, the `name` variable must be passed to wrapper.

In [ ]:
def my_decorator(func):
    def wrapper(name):
        print("I come before the function!")
        func(name)
        print("I come after the function!")
    return wrapper

my_hello_func = my_decorator(say_hello)

In [ ]:
my_hello_func('Ivan')

After passing the name variable, the code works properly. Observe that in the last call, `Ivan` is passed as an argument. This is because `my_hello_func` is the `wrapper` with information on `say_hello`, and `wrapper` requires an argument.

Decorators enable us to pass arguments to the function. For example, a name can be passed to the function and subsequently employed to greet the user. As a better example, we can measure the time it takes for a function to execute, as shown below.

In [ ]:
import time

def my_timer(func):
    def wrapper():
        time_0 = time.time()
        func()
        time_1 = time.time()
        print(f'It took {time_1 - time_0} second to run')
    return wrapper


def dummy_fun():
    for _ in range(50000000):
        x = 'I am just losing your time'
    return x

time_exec = my_timer(dummy_fun)
print(time_exec)

Observe what happens when the function is called:

In [ ]:
time_exec()

As 'syntactic sugar', it is possible to decorate a function by adding `@` when defining the function.

In [ ]:
@my_timer
def dummy_fun():
    for _ in range(50000000):
        x = 'I am just losing your time'

dummy_fun()

## Multiple Arguments in a Decorator

We have established that the functionality of a function can be extended using decorators. Since we have no control over the functions to be wrapped by the decorator, we can employ *args **kwargs to ensure that it applies to functions with multiple arguments.

In [ ]:
def repeat(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper

def say_hi(name):
    print(f'Hello {name}')
    return 1

repeat(say_hi)('Ivan')

## `Return` Statement in the Wrapper

Here, we discuss how to return information from a decorated function.

We define a small function: factorial, which returns the factorial of a number (the factorial of a number is the product of all the numbers from 1 to that number).

In [ ]:
def factorial(n):
    previous = 1
    for i in range(1, n + 1):
        previous *= i
    return previous

print(factorial(5))

Now, we define a timer decorator. However, in this case, the decorated function can accept any argument. To account for this, we employ `*args` and `**kwargs`.

In [ ]:
def my_timer(fun):
    def wrapper(*args, **kwargs):
        time_0 = time.time()
        fun(*args, **kwargs)
        time_1 = time.time()
        print(f'It took {time_1 - time_0} second to run')
    return wrapper


@my_timer
def factorial(n):
    previous = 1
    for i in range(1, n + 1):
        previous *= i
    return previous

print(factorial(5))

Although the message detailing the execution duration was successfully printed, the value of the factorial was not printed. That is because wrapper does not return anything, hence the 'None' output. As a solution, we simply introduce a `return` statement.

In [ ]:
def my_timer(fun):
    def wrapper(*args, **kwargs):
        time_0 = time.time()
        output = fun(*args, **kwargs)
        time_1 = time.time()
        print(f'It took {time_1 - time_0} second to run')
        return output
    return wrapper


@my_timer
def factorial(n):
    previous = 1
    for i in range(1, n + 1):
        previous *= i
    return previous

print(factorial(5))

## Classmethods and Staticmethods

Two often-used decorators are classmethods and staticmethods. They add functionality to methods

Classmethods must have a reference to a class object instead of an instance, whereas static methods do not point to an instance or a class. Note that classmethods are bound to the class; thus, they must be called from that class or from an instance of that class.

Consider the example below, where the classmethod and staticmethod are applied. The class, `Date`, is defined, which is initialised with three parameters: day, month and year. It also has a method, `get_date`, that returns the date in a string format.

In [ ]:
class Date:

    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

    def get_date(self):
        print(f'The date is {self.day}/{self.month}')

date = Date(19, 5, 1991)
date.get_date()

Subsequently, a classmethod is added to `Date`.

> <font size=+1>Classmethods are used to create methods that do not depend on the instance but the class.</font>

This indicates that a classmethod can be called without an instance of the class. Actually, a classmethod is called before the `__init__` method.

As an application, classmethods can be used to create an instance of `Date` that directly provides the current date. This can be achieved by calling a classmethod that returns the current date, and no input is required.

The syntax of a classmethod is
```
@classmethod
def method_name(cls, *args, **kwargs):
    # do something
    return something
```

Important! Notice that the first argument of the classmethod is the class itself, not an instance of the class. Therefore, '`cls`' is used instead of '`self`'.

In [ ]:
import datetime

class Date:

    def __init__(self, day=0, month=0, year=0):
        print('__init__ method called!')
        self.day = day
        self.month = month
        self.year = year

    def get_date(self):
        print(f'The date is {self.day}/{self.month}')

    @classmethod
    def today(cls):
        print('Classmethod called!')
        today = datetime.date.today()
        day = today.day
        month = today.month
        year = today.year
        print('I am about to call the __init__ method')
        return cls(day, month, year)

Observe the code above (a few print statements have been added to enhance comprehension). A classmethod that returns the current date has been created. The method initially calculates the current date and subsequently calls the constructor of the class, `Date`, with the arguments we calculated.

We instansiate a class, `Date`, using its classmethod as follows (observe what is printed and compare the output while following the flow of the code):

In [ ]:
current_date = Date.today()

In [ ]:
current_date.get_date()

### Staticmethods

> <font size=+1>Staticmethods are methods that are not bound to an instance or a class.</font>

Thus, the '`self`' and '`cls`' arguments are not required because any data passed to the staticmethod will not depend on the instance or the class.

Similar to classmethods, staticmethods can be defined as follows:
```
@staticmethod
def method_name(*args, **kwargs):
    # do something
    return something
```

Here, we define a staticmethod that returns checks if the date is valid. Staticmethods are perfect for this task, since determining if a date is valid does not depend on the instance or the class.

In [ ]:
class Date:

    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

    def get_date(self):
        print(self.day, self.month)

    @classmethod
    def today(cls):
        today = datetime.date.today()
        day = today.day
        month = today.month
        year = today.year
        return cls(day, month, year)

    @staticmethod
    def is_date_valid(day, month, year):
        return day <= 31 and month <= 12 and year <= 3999

Once again, staticmethods do not require the `self` or `cls` argument, indicating that they can be called from an instance or the class.

In [ ]:
Date.is_date_valid(1, 1, 1)

In [ ]:
date = Date.today()
date.is_date_valid(1, 1, 1)

As an added benefit, the staticmethod can be used inside the `__init__` method of the class `Date`; thus, when the arguments are passed to the constructor, the staticmethod can be utilised to determine the validity of the date.

In [ ]:
class Date:

    def __init__(self, day=0, month=0, year=0):
        if self.is_date_valid(day, month, year):
            self.day = day
            self.month = month
            self.year = year
        else:
            raise ValueError('Invalid date!')

    def get_date(self):
        print(self.day, self.month)

    @classmethod
    def today(cls):
        today = datetime.date.today()
        day = today.day
        month = today.month
        year = today.year
        return cls(day, month, year)

    @staticmethod
    def is_date_valid(day, month, year):
        return day <= 31 and month <= 12 and year <= 3999

If a valid date is passed, the instance will be created, without any error. Conversely, if an invalid date is passed, Python will throw an error (ValueError).

In [ ]:
good_date = Date(19, 5, 1991)
# No error thrown, and the code works.

In [ ]:
bad_date = Date(31, 13, 2021)

## Conclusion
In this lesson, we reviewed decorators on the surface level. To improve your knowledge and understanding of decorators, engage in practicals and participate in as many challenges as possible. For examples, see [here](https://github.com/IvanYingX/Challenges_AiCore.git)